In [1]:
import numpy as np 
import pandas as pd 

# To consume Twitter's API
import tweepy
from tweepy import OAuthHandler 

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
# keys and tokens from the Twitter Dev Console
consumer_key = 'Sec3MvclRIx2RVlgu9l0SJX6D'
consumer_secret = 'ayoPNWtBm7fWpMBoK6EwRmegu3SW8Rw9mzJkottkv97quPe941'
access_token = '736550752760406018-so5CPJrEbJKb3c3Pq8va3VFr0yk4S0E'
access_token_secret = 'Cgr8tz0h6FTU7kxAjDzpHnjffNTHxWsBytXnu4Ihd1TFb'

In [3]:
class TwitterClient(object): 
    def __init__(self): 
        #Initialization method. 
        try: 
            # create OAuthHandler object 
            auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            # add hyper parameter 'proxy' if executing from behind proxy "proxy='http://172.22.218.218:8085'"
            self.api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
            
        except tweepy.TweepError as e:
            print(f"Error: Tweeter Authentication Failed - \n{str(e)}")

    def get_tweets(self, query, maxTweets = 1000):
        #Function to fetch tweets. 
        # empty list to store parsed tweets 
        tweets = [] 
        sinceId = None
        max_id = -1
        tweetCount = 0
        tweetsPerQry = 100

        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = self.api.search(q=query, count=tweetsPerQry)
                    else:
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                since_id=sinceId)
                else:
                    if (not sinceId):
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                max_id=str(max_id - 1))
                    else:
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                max_id=str(max_id - 1),
                                                since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break

                for tweet in new_tweets:
                    parsed_tweet = {} 
                    parsed_tweet['tweets'] = tweet.text 

                    # appending parsed tweet to tweets list 
                    if tweet.retweet_count > 0: 
                        # if tweet has retweets, ensure that it is appended only once 
                        if parsed_tweet not in tweets: 
                            tweets.append(parsed_tweet) 
                    else: 
                        tweets.append(parsed_tweet) 
                        
                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id

            except tweepy.TweepError as e:
                # Just exit if any error
                print("Tweepy error : " + str(e))
                break
        
        return pd.DataFrame(tweets)

## (A). Mining Latest trending topic for India from twitter

In [4]:
twitter_client = TwitterClient()

# calling function to get tweets
tweets_df = twitter_client.get_tweets('Trending India', maxTweets=100)
print(f'tweets_df Shape - {tweets_df.shape}')
tweets_df.head(10)

Downloaded 100 tweets
tweets_df Shape - (24, 1)


,tweets
0,RT @rishibagree: #NathuramGodse is a top trend...
1,RT @MlaGorakhnath: THANK YOU EVERY NATIONALIST...
2,RT @SuryaFansRage: This is pure Mass Daily #Su...
3,RT @Suriya40TheFilm: Now #Suriya40 Trending In...
4,RT @dineshbjp09: THANK YOU EVERY NATIONALIST 🇮...
5,"RT @jaspreetkaurg_: the fact that ""OperationBl..."
6,RT @TeamAlyGoni: YAY TRENDING IN INDIA.\n\n#Al...
7,RT @SuryaFansAndhra: Breaking :) @Suriya_offl\...
8,RT @SuruliPasubathi: #Dhanush\n#JagameThandhir...
9,RT @seokjin_india_: [#JinTrend] India 🇮🇳\n\nSe...


In [5]:
df = tweets_df
df.to_csv('Trending India.csv') 

## (B). Mining 100+ tweets for #JoeBiden from Twitter 

In [6]:
twitter_client = TwitterClient()

# calling function to get tweets
tweets_df = twitter_client.get_tweets('#JoeBiden', maxTweets=100)
print(f'tweets_df Shape - {tweets_df.shape}')
tweets_df.head(10)

Downloaded 83 tweets
Downloaded 179 tweets
tweets_df Shape - (142, 1)


,tweets
0,RT @EmmanuelChaunu: Publié aujourd’hui dans @o...
1,RT @elliielle: Dear President @POTUS and @VP...
2,RT @KatholikenNet: #biden #joebiden https://t....
3,RT @kingron_b: It's that time of the day! Acco...
4,Presidents @Mbuhari &amp; @Joebiden all signed...
5,RT @Clarsonimus: Which form of cognitive disor...
6,"RT @robblue22: King Biden and His Magic Pen, w..."
7,#farmersrprotest #Punjabi_GeorgeFloyd #Farme...
8,RT @DainikBhaskar: US चीन टकराव: अमेरिका ने कह...
9,RT @bartolopietro1: Abbiamo di nuovo buoni ami...


In [7]:
df = tweets_df
df.to_csv('Joe Biden.csv') 

## (C) Sentiment Analysis of Joe Biden Tweets

In [8]:
def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    return 'neg' if polarity_scores['neg'] > polarity_scores['pos'] else 'pos'

In [9]:
sentiments_using_SIA = tweets_df.tweets.apply(lambda tweet: fetch_sentiment_using_SIA(tweet))
pd.DataFrame(sentiments_using_SIA.value_counts())

,tweets
pos,113
neg,29


In [10]:
tweets_df['sentiment'] = sentiments_using_SIA
tweets_df.head()

,tweets,sentiment
0,RT @EmmanuelChaunu: Publié aujourd’hui dans @o...,pos
1,RT @elliielle: Dear President @POTUS and @VP...,pos
2,RT @KatholikenNet: #biden #joebiden https://t....,pos
3,RT @kingron_b: It's that time of the day! Acco...,pos
4,Presidents @Mbuhari &amp; @Joebiden all signed...,pos


In [11]:
df = tweets_df
df.to_csv('Joe Biden with sentiment.csv') 